In [1]:
import requests

from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq

In [2]:
model = AutoModelForVision2Seq.from_pretrained("microsoft/kosmos-2-patch14-224")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.45k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.66G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/191k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

In [3]:

prompt = """<grounding>"Describe the image in detail, characters, text, emotions, gestures, interactions, and any offensive or harmful elements revealing cloths, humor, sarcasm, stereotypes, or demeaning content, including all objects, text, expressions, gestures, interactions, and any offensive or hateful elements."</grounding>"""

# Use a local image
image_path = "/content/download.jpg"
image = Image.open(image_path)

# Save and reload the image to ensure consistency
image.save("new_image.jpg")
image = Image.open("new_image.jpg")

inputs = processor(text=prompt, images=image, return_tensors="pt")

generated_ids = model.generate(
    pixel_values=inputs["pixel_values"],
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    image_embeds=None,
    image_embeds_position_mask=inputs["image_embeds_position_mask"],
    use_cache=True,
    max_new_tokens=128,
)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

processed_text = processor.post_process_generation(generated_text, cleanup_and_extract=True)

print(processed_text)




('"Describe the image in detail, characters, text, emotions, gestures, interactions, and any offensive or harmful elements revealing cloths, humor, sarcasm, stereotypes, or demeaning content, including all objects, text, expressions, gestures, interactions, and any offensive or hateful elements." The image shows a young girl smiling in front of a burning house, with a fire truck and firefighters in the background. The girl is wearing a red bow, and the fire is visible in the house.', [('a young girl', (312, 324), [(0.546875, 0.203125, 0.984375, 0.984375)]), ('a burning house', (345, 360), [(0.015625, 0.171875, 0.546875, 0.546875)]), ('a fire truck', (367, 379), [(0.484375, 0.046875, 0.796875, 0.515625)]), ('firefighters', (384, 396), [(0.390625, 0.421875, 0.453125, 0.671875), (0.515625, 0.390625, 0.546875, 0.578125)]), ('a red bow', (436, 445), [(0.546875, 0.515625, 0.609375, 0.609375)])])


In [4]:
# Extract the text from the tuple if necessary
if isinstance(processed_text, tuple):
    description = processed_text[0]  # Get the first element
else:
    description = processed_text  # Use directly if it's already a string


if '"' in description:
    description = description.split('"', 2)[-1].strip()


print(description)

The image shows a young girl smiling in front of a burning house, with a fire truck and firefighters in the background. The girl is wearing a red bow, and the fire is visible in the house.
